
Entailment


In [3]:
pip install transformers

     |████████████████████████████████| 1.4MB 8.2MB/s 
     |████████████████████████████████| 2.9MB 15.9MB/s 
     |████████████████████████████████| 890kB 55.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=ba61a9d0986391ac87b5bfe6a9e35345622ee824fdf719d4dd916758ae0196e7
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [1]:
pip install sentencepiece

     |████████████████████████████████| 1.1MB 6.8MB/s 


In [4]:
import pandas as pd

In [5]:
from google.colab import files 
  
  
uploaded = files.upload()

Saving cisco_sample.xlsx to cisco_sample.xlsx


In [6]:
import pandas as pd 
import io 
  
df = pd.read_excel(io.BytesIO(uploaded['cisco_sample.xlsx'])) 
print(df)

                            Product  ...                                           Question
0                        Cisco 9400  ...   what are the features of  Catalyst 9400 Series ?
1                      Cisco 9400    ...    what is the warrenty period for Catalyst 9400 ?
2                        Cisco 8500  ...  Advantage of Catalyst 9300 Series over the pre...
3                        Cisco 8500  ...  How to get access to all Catalyst 8500 platfor...
4                   8841 TSG Phones  ...      What are the Key features of 8841 TSG Phones?
5   8841 cucm 12.5 activation codes  ...           what is 8841 cucm 12.5 activation codes?
6                   8841 TSG Phones  ...                                  what is 8841 TSG?
7                   8841 TSG Phones  ...  What are the Advantage of 8841 TSG Over pervio...
8                   8841 TSG Phones  ...           How to delete itl file cisco phone 8841?
9                      Cisco Phones  ...                      How do I mute my C

In [ ]:
df.columns

Index(['Product', 'Category', 'Solution', 'Link', 'Question'], dtype='object')

In [7]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

max_length = 256


#hg_model_hub_name = "ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli"
hg_model_hub_name = "ynie/albert-xxlarge-v2-snli_mnli_fever_anli_R1_R2_R3-nli"
# hg_model_hub_name = "ynie/bart-large-snli_mnli_fever_anli_R1_R2_R3-nli"
# hg_model_hub_name = "ynie/electra-large-discriminator-snli_mnli_fever_anli_R1_R2_R3-nli"
# hg_model_hub_name = "ynie/xlnet-large-cased-snli_mnli_fever_anli_R1_R2_R3-nli"

tokenizer = AutoTokenizer.from_pretrained(hg_model_hub_name)
model = AutoModelForSequenceClassification.from_pretrained(hg_model_hub_name)



premise = str(input('Premise: '))
confidence_limit= 0.1
relevant_que= []

for i in range(len(df['Question'])):
  hypothesis= str(df.iloc[i,4])
  tokenized_input_seq_pair = tokenizer.encode_plus(premise, hypothesis,
                                                     max_length=max_length,
                                                     return_token_type_ids=True, truncation=True)
  
  input_ids = torch.Tensor(tokenized_input_seq_pair['input_ids']).long().unsqueeze(0)
  token_type_ids = torch.Tensor(tokenized_input_seq_pair['token_type_ids']).long().unsqueeze(0)
  attention_mask = torch.Tensor(tokenized_input_seq_pair['attention_mask']).long().unsqueeze(0)

  outputs = model(input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids,labels=None)

  predicted_probability = torch.softmax(outputs[0], dim=1)[0].tolist()

  #print("Premise:", premise)
  #print("Hypothesis:", hypothesis)
  #print("Entailment:", predicted_probability[0])
  #print("Neutral:", predicted_probability[1])
  #print("Contradiction:", predicted_probability[2])
  if predicted_probability[0] >= confidence_limit:
    relevant_que.append(hypothesis)
  
print('Hpyothesis:')  
print('\n'.join(relevant_que))

 # Note:
    # "id2label": {
    #     "0": "entailment",
    #     "1": "neutral",
    #     "2": "contradiction"
    # },

  
  


Premise: what is catalyst 9400
Hpyothesis:
what are the features of  Catalyst 9400 Series ?
what is 8841 TSG?
